## Feature extraction 

In [3]:
import pandas as pd
X_train = pd.read_csv("/code/data/train.csv")['headline']
y_train = pd.read_csv("/code/data/train.csv")['is_sarcastic']

X_val = pd.read_csv("/code/data/val.csv")['headline']
y_val = pd.read_csv("/code/data/val.csv")['is_sarcastic']

In [4]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Initialize the `tfidf_vectorizer` 
#tfidf_vectorizer = TfidfVectorizer(stop_words='english') 
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the val set 
tfidf_val = tfidf_vectorizer.transform(X_val)

## Classifiers 

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn import metrics

# clf1 = MultinomialNB() 

# Predict bag of words

# clf1.fit(tfidf_train, y_train)
# pred1 = clf1.predict(tfidf_val)
# metrics.accuracy_score(y_val, pred1)

In [ ]:
# from sklearn import svm

# clf2 = svm.SVC(gamma='scale', probability=True)

# clf2.fit(tfidf_train, y_train)
# pred2 = clf2.predict(tfidf_val)
# metrics.accuracy_score(y_val, pred2)

In [ ]:
# from itertools import product
# from sklearn.ensemble import VotingClassifier
# # Loading some example data

# eclf = VotingClassifier(estimators=[('mnb', clf1), ('svm', clf2)],
#                         voting='soft', weights=[1, 1])
# # clf1 = clf1.fit(X, y)
# # clf2 = clf2.fit(X, y)
# eclf = eclf.fit(tfidf_train, y_train)
# pred3 = eclf.predict(tfidf_val)
# metrics.accuracy_score(y_val, pred3)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

clf1 = LogisticRegression(multi_class='multinomial', random_state=1,  solver='newton-cg')
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
# clf3 = GaussianNB()
# clf4 = svm.SVC(gamma='scale', probability=True)
clf5 = MultinomialNB() 


In [17]:
from sklearn.ensemble import VotingClassifier
from itertools import combinations_with_replacement
from sklearn import metrics

results = [[0],[0,0,0]]
comb = combinations_with_replacement([0,1,2,3,4],3)
next(comb)
for i, combination in enumerate(comb):
    print(combination)
    
    eclf = VotingClassifier(estimators=[('clf2', clf2),('clf1', clf1), ('clf5', clf5)],
                            voting='soft', weights=combination)
    # clf1 = clf1.fit(X, y)
    # clf2 = clf2.fit(X, y)
    eclf = eclf.fit(tfidf_train, y_train)
    pred3 = eclf.predict(tfidf_val)
    result = metrics.accuracy_score(y_val, pred3)
    print(result)
    results[0].append(result)
    results[1].append(combination)


(0, 0, 1)
0.8322851153039832
(0, 0, 2)


KeyboardInterrupt: 

In [12]:
from sklearn import metrics
pred3 = eclf.predict(tfidf_val)
print(metrics.accuracy_score(y_val, pred3))

0.8551129746098299
